## 3) 주어진 호텔 리뷰 데이터를 가지고 할 수 있는 분석이 있으면 더 진행해보시오.

In [ ]:
from string import punctuation
from tensorflow.keras.utils import to_categorical
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, SimpleRNN
from google.colab import drive

### 자연어 생성
리뷰들을 'Hotel_reviews_20k.csv' 파일에서 가져온 후 RNN을 이용하여 모델을 학습 시킨 다음 좋은 리뷰와 나쁜 리뷰를 생성했다

In [ ]:
drive.mount('/content/drive')

# CSV 파일을 pandas 데이터프레임으로 로드
df = pd.read_csv('Hotel_reviews_20k.csv', encoding='latin1')

In [ ]:
Good = df.loc[df['rating_review']== 5 ]
Good = Good['review_full'][:10]
Good = Good.values

def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

Good_s = [repreprocessing(x) for x in Good]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(Good_s)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

sequences = list()
for line in Good_s: # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=70,batch_size=32, verbose=2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
단어 집합의 크기 : 606
학습에 사용할 샘플의 개수: 1461
샘플의 최대 길이 : 635
Epoch 1/70
46/46 - 39s - loss: 6.3213 - accuracy: 0.0479 - 39s/epoch - 858ms/step
Epoch 2/70
46/46 - 34s - loss: 5.9185 - accuracy: 0.0657 - 34s/epoch - 742ms/step
Epoch 3/70
46/46 - 37s - loss: 5.7108 - accuracy: 0.0657 - 37s/epoch - 794ms/step
Epoch 4/70
46/46 - 32s - loss: 5.6812 - accuracy: 0.0657 - 32s/epoch - 706ms/step
Epoch 5/70
46/46 - 32s - loss: 5.6732 - accuracy: 0.0657 - 32s/epoch - 688ms/step
Epoch 6/70
46/46 - 31s - loss: 5.6690 - accuracy: 0.0657 - 31s/epoch - 677ms/step
Epoch 7/70
46/46 - 33s - loss: 5.6669 - accuracy: 0.0657 - 33s/epoch - 707ms/step
Epoch 8/70
46/46 - 33s - loss: 5.6644 - accuracy: 0.0657 - 33s/epoch - 712ms/step
Epoch 9/70
46/46 - 34s - loss: 5.6600 - accuracy: 0.0657 - 34s/epoch - 730ms/step
Epoch 10/70
46/46 - 31s - loss: 5.6513 - accuracy: 0.0657 - 31s/epoch - 684ms/st

In [ ]:
X.shape

(1461, 634)

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

- 위의 코드는 좋은 리뷰를 모델에게 학습시키는 코드이다
- 학습시킨 모델을 이용해서 자연어를 생성한다

In [ ]:
print(sentence_generation(model, tokenizer, 'hotel', 15))
print(sentence_generation(model, tokenizer, 'This hotel', 20))
print(sentence_generation(model, tokenizer, 'First visit', 15))

hotel is very good with it up motifs and the meal and its immediately recognizable though
This hotel the best of the best adores noodle soups and all bite and hot and complimented a little but to is
First visit jamun and very crispy and hot and assorted good with it had food the best


- 시작 단어를 hotel, This hotel, First visit과 같이 주었는데 어느 정도 좋은 리뷰들을 잘 조합해내는 것을 볼 수 있다
- epoch를 70으로 batch size는 32로 학습시키는데 무려 40분이 소요되었다
- 만약 epoch를 더 늘리고, batch size를 줄여서 학습 시키면 더 연관성 있는 리뷰를 생성할 것이라고 예상할 수 있다

In [ ]:
# 부정적리뷰
Bad = df.loc[df['rating_review']== 1 ]
Bad = Bad['review_full'][20:40]
Bad = Bad.values

def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

Bad_s = [repreprocessing(x) for x in Bad]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(Bad_s)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

sequences_b = list()
for line in Bad_s: # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences_b.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences_b))

max_len = max(len(l) for l in sequences_b) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

sequences_b = pad_sequences(sequences_b, maxlen=max_len, padding='pre')
sequences_b = np.array(sequences_b)
X_b = sequences_b[:,:-1]
y_b = sequences_b[:,-1]
y_b = to_categorical(y_b, num_classes=vocab_size)

embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_b, y_b, epochs=30,batch_size=32, verbose=2)

단어 집합의 크기 : 757
학습에 사용할 샘플의 개수: 2106
샘플의 최대 길이 : 382
Epoch 1/70
66/66 - 42s - loss: 6.4372 - accuracy: 0.0171 - 42s/epoch - 642ms/step
Epoch 2/70
66/66 - 38s - loss: 5.9205 - accuracy: 0.0484 - 38s/epoch - 579ms/step
Epoch 3/70
66/66 - 35s - loss: 5.8436 - accuracy: 0.0484 - 35s/epoch - 528ms/step
Epoch 4/70
66/66 - 33s - loss: 5.8329 - accuracy: 0.0484 - 33s/epoch - 495ms/step
Epoch 5/70
66/66 - 31s - loss: 5.8290 - accuracy: 0.0484 - 31s/epoch - 467ms/step
Epoch 6/70
66/66 - 30s - loss: 5.8243 - accuracy: 0.0484 - 30s/epoch - 459ms/step
Epoch 7/70
66/66 - 30s - loss: 5.8191 - accuracy: 0.0484 - 30s/epoch - 460ms/step
Epoch 8/70


In [ ]:
print(sentence_generation(model, tokenizer, 'hotel', 15))
print(sentence_generation(model, tokenizer, 'this hotel', 20))
print(sentence_generation(model, tokenizer, 'visit', 15))
print(sentence_generation(model, tokenizer, 'it', 15))
print(sentence_generation(model, tokenizer, 'the', 15))

hotel my mutton pm me may hamper a few times and half hours that the oven
this hotel that a be pm from may hamper dont restaurant and found dont embarrassing to send behave customer when and were
visit that a few minutes of were food i am my manager offered in 20 minutes
it the mutton quality bit high market dimension solution ltd reads my ltd offered in a
the mutton i sold may hamper a few minutes of were food i am my manager


### 결론
- 시작 단어를 hotel, this hotel, visit, it, the 와 같이 주었는데 어느 정도 나쁜 리뷰들을 잘 조합해내는 것을 볼 수 있다
- epoch를 30으로 batch size는 32로 학습시키는데 무려 20분이 소요되었다
- 만약 epoch를 더 늘리고, batch size를 줄여서 학습 시키면 더 연관성 있는 리뷰를 생성할 것이라고 예상할 수 있다